# [Redis LangChain OpenAI eCommerce Chatbot](https://lablab.ai/t/redis-langchain-ecommerce-chatbot) (lablab.ai)

In [ ]:
# Install requirements
# %pip install -r requirements.txt

## Loading and Preprocessing the Data

In [ ]:
import numpy as np
import pandas as pd
import time
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query
import redis


redis_conn = redis.Redis(
    host="redis-18975.c300.eu-central-1-1.ec2.cloud.redislabs.com",
    port=18975,
    password="xxxxxxxxxxx",
)

In [ ]:
MAX_TEXT_LENGTH = 512
NUMBER_PRODUCTS = 1000


def auto_truncate(val):
    return val[:MAX_TEXT_LENGTH]


# Load Product data and truncate long text fields
all_prods_df = pd.read_csv(
    "product_data.csv",
    converters={
        "bullet_point": auto_truncate,
        "item_keywords": auto_truncate,
        "item_name": auto_truncate,
    },
)
all_prods_df["primary_key"] = (
    all_prods_df["item_id"] + "-" + all_prods_df["domain_name"]
)
all_prods_df["item_keywords"].replace("", np.nan, inplace=True)
all_prods_df.dropna(subset=["item_keywords"], inplace=True)
all_prods_df.reset_index(drop=True, inplace=True)

# get the first 1000 products with non-empty item keywords
product_metadata = all_prods_df.head(NUMBER_PRODUCTS).to_dict(orient="index")

all_prods_df.head()

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-distilroberta-v1")


item_keywords = [product_metadata[i]["item_keywords"] for i in product_metadata.keys()]
item_keywords_vectors = [model.encode(sentence) for sentence in item_keywords]

In [ ]:
len(item_keywords_vectors)
len(product_metadata)

# Check one of the products
product_metadata[0]

## Creating the Redis Index and Loading Vectors

Create a function to load vectors into the Redis index and a function to create a flat index. We will use these functions later to index our product data.

In [ ]:
def load_vectors(client, product_metadata, vector_dict, vector_field_name):
    p = client.pipeline(transaction=False)
    for index in product_metadata.keys():
        # hash key
        key = "product:" + str(index) + ":" + product_metadata[index]["primary_key"]

        # hash values
        item_metadata = product_metadata[index]
        item_keywords_vector = vector_dict[index].astype(np.float32).tobytes()
        item_metadata[vector_field_name] = item_keywords_vector

        # HSET
        p.hset(key, mapping=item_metadata)

    p.execute()


def create_flat_index(
    redis_conn,
    vector_field_name,
    number_of_vectors,
    vector_dimensions=512,
    distance_metric="L2",
):
    redis_conn.ft().create_index(
        [
            VectorField(
                vector_field_name,
                "FLAT",
                {
                    "TYPE": "FLOAT32",
                    "DIM": vector_dimensions,
                    "DISTANCE_METRIC": distance_metric,
                    "INITIAL_CAP": number_of_vectors,
                    "BLOCK_SIZE": number_of_vectors,
                },
            ),
            TagField("product_type"),
            TextField("item_name"),
            TextField("item_keywords"),
            TagField("country"),
        ]
    )

## Connect to our Redis DB

In [ ]:
ITEM_KEYWORD_EMBEDDING_FIELD = "item_keyword_vector"
TEXT_EMBEDDING_DIMENSION = 768
NUMBER_PRODUCTS = 1000

print("Loading and Indexing + " + str(NUMBER_PRODUCTS) + " products")

# flush all data
redis_conn.flushall()

# create flat index & load vectors
create_flat_index(
    redis_conn,
    ITEM_KEYWORD_EMBEDDING_FIELD,
    NUMBER_PRODUCTS,
    TEXT_EMBEDDING_DIMENSION,
    "COSINE",
)
load_vectors(
    redis_conn, product_metadata, item_keywords_vectors, ITEM_KEYWORD_EMBEDDING_FIELD
)

## Creating the Chatbot

### Go from user input string to product keywords

In [ ]:
import os
import dotenv

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# loads .env file with your OPENAI_API_KEY
dotenv.load_dotenv()


# Define embedding model
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.3, openai_api_key=OPENAI_API_KEY)
prompt = PromptTemplate(
    input_variables=["product_description"],
    template="Create comma seperated product keywords to perform a query on a amazon dataset for this user input: {product_description}",
)

from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
userinput = input("Hey im a E-commerce Chatbot, how can i help you today? ")
print("User:", userinput)

# Run the chain only specifying the input variable.
keywords = chain.run(userinput)

### Query our data

Use the generated keywords to query the product embeddings in Redis and retrieve the top 3 results.

In [ ]:
top_k = 3
# Vectorize the query
query_vector = model.encode(keywords).astype(np.float32).tobytes()

# Prepare the query
q = (
    Query(
        f"*=>[KNN {top_k} @{ITEM_KEYWORD_EMBEDDING_FIELD} $vec_param AS vector_score]"
    )
    .sort_by("vector_score")
    .paging(0, top_k)
    .return_fields("vector_score", "item_name", "item_id", "item_keywords")
    .dialect(2)
)
params_dict = {"vec_param": query_vector}

# Execute the query
results = redis_conn.ft().search(q, query_params=params_dict)

full_result_string = ""
for product in results.docs:
    full_result_string += (
        product.item_name
        + " "
        + product.item_keywords
        + " "
        + product.item_id
        + "\n\n\n"
    )

### Create the chatbot

In [ ]:
from langchain.memory import ConversationBufferMemory


template = """You are a chatbot. Be kind, detailed and nice. Present the given queried search result in a nice way as answer to the user input. dont ask questions back! just take the given context

{chat_history}
Human: {user_msg}
Chatbot:"""

prompt = PromptTemplate(input_variables=["chat_history", "user_msg"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    llm=OpenAI(
        model_name="gpt-3.5-turbo", temperature=0.8, openai_api_key=OPENAI_API_KEY
    ),
    prompt=prompt,
    verbose=False,
    memory=memory,
)

answer = llm_chain.predict(user_msg=f"{full_result_string} ---\n\n {userinput}")
print("Bot:", answer)
time.sleep(0.5)

while True:
    follow_up = input("Anything else you want to ask about this topic?")
    print("User:", follow_up)
    answer = llm_chain.predict(user_msg=follow_up)
    print("Bot:", answer)
    time.sleep(0.5)